In [ ]:
! pip install kaggle



In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
#API to fetch the dataset from Kaggle
! Kaggle

/bin/bash: line 1: Kaggle: command not found


In [ ]:
! kaggle datasets download -d Kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/Kazanova/sentiment140
License(s): other
 95% 77.0M/80.9M [00:00<00:00, 165MB/s]
100% 80.9M/80.9M [00:00<00:00, 148MB/s]


In [ ]:
#Extracting the compressed data

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


Importing the libraries


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#print the stopwords in English
print(stopwords.words('english'))

In [ ]:
# loading the data from csv to pandas dataframe

twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [ ]:
#twitter data: checking rows and columns
twitter_data.shape

(1600000, 6)

In [ ]:
# name the colums
columns = ['target','ids','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names = columns, encoding='ISO-8859-1')

In [ ]:
# check if any values missing

twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
# checking the label distribuiton
twitter_data['target'].value_counts()


,count
target,
0,800000
1,800000


Convert Target 4 to 1

In [ ]:
twitter_data['target'] = twitter_data['target'].replace(4,1)

0 --> Negative Tweet
1 --> Postive Tweet

Stemming is the process of reducing a word to its Root Words, Example: words like: actor, actress, acting = act

**Stemming**

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content
#

In [ ]:
twitter_data['text'] = twitter_data['text'].apply(stemming)

In [ ]:
X = twitter_data['text'].values
Y = twitter_data['target'].values

**Spliting data in train and Test**

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)


In [ ]:
vectorizer = TfidfVectorizer()


X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


In [ ]:
model = LogisticRegression(max_iter=2000)




In [ ]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=2000)

**Model Evaluation**


In [ ]:
#accuracy score om training data
x_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(x_train_prediction, Y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.79871953125


In [ ]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.77668125


In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
model = pickle.load(open(filename, 'rb'))

In [ ]:
new_tweet =  "I'm tireD"
new_tweet_vectorized = vectorizer.transform([new_tweet])  # Notice it's a list with a single string

# Step 2: Predict sentiment using your trained model
prediction = model.predict(new_tweet_vectorized)

# Step 3: Interpret the prediction
# If it's a binary classification, prediction might return 0 (negative) or 1 (positive)
if prediction == 1:
    print("The tweet is positive!")
else:
    print("The tweet is negative!")


The tweet is negative!


In [ ]:
import joblib

# Save the vectorizer
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']